# Preprocessing: BJH029


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH029'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH029/ieeg/BJH029_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH029/ieeg/BJH029_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 3059399 =   1043.000 ...  1529.699 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH029/ieeg/BJH029_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH029/ieeg/BJH029_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 3059399 =   1043.000 ...  1529.699 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

EMPTY noref
EMPTY_2 noref
EMPTY_3 noref
EMPTY_4 noref
REF1 noref
REF2 noref
A'1 A'2
A'2 A'3
A'3 A'4
A'4 A'5
A'5 A'6
A'6 A'7
A'7 A'8
A'8 A'9
A'9 A'10
A'10 A'11
A'11 A'12
A'12 A'13
A'13 A'14
B'1 B'2
B'2 B'3
B'3 B'4
B'4 B'5
B'5 B'6
B'6 B'7
B'7 B'8
B'8 B'9
B'9 B'10
B'10 B'11
B'11 B'12
C'1 C'2
C'2 C'3
C'3 C'4
C'4 C'5
C'5 C'6
C'6 C'7
C'7 C'8
D'1 D'2
D'2 D'3
D'3 D'4
D'4 D'5
D'5 D'6
D'6 D'7
D'7 D'8
E'1 E'2
E'2 E'3
E'3 E'4
E'4 E'5
E'5 E'6
E'6 E'7
E'7 E'8
E'8 E'9
E'9 E'10
EMPTY_59 noref
EMPTY_60 noref
EMPTY_61 noref
EMPTY_62 noref
EMPTY_63 noref
EMPTY_64 noref
F'1 F'2
F'2 F'3
F'3 F'4
F'4 F'5
F'5 F'6
F'6 F'7
F'8 noref
G'1 G'2
G'2 G'3
G'3 G'4
G'4 G'5
G'5 G'6
G'6 G'7
G'7 G'8
G'8 G'9
G'9 G'10
G'10 G'11
G'11 G'12
G'12 G'13
G'13 G'14
G'14 G'15
G'15 G'16
H'1 H'2
H'2 H'3
H'3 H'4
H'4 H'5
H'5 H'6
H'6 H'7
H'7 H'8
H'8 H'9
H'9 H'10
H'10 H'11
H'11 H'12
I1 I2
I2 I3
I3 I4
I4 I5
I5 I6
I6 I7
I7 I8
I8 I9
I9 I10
I10 I11
I11 I12
I12 I13
I13 I14
J'1 J'2
J'2 J'3
J'3 J'4
J'4 J'5
J'5 J'6
J'6 J'7
J'7 J'8
J'8 J'9
J'9 J'10

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 3059399  =      0.000 ...  1529.699 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=173, n_times=3059400
    Range : 0 ... 3059399 =      0.000 ...  1529.699 secs
Ready.
Added the following bipolar channels:
A'1-A'2, A'2-A'3, A'3-A'4, A'4-A'5, A'5-A'6, A'6-A'7, A'7-A'8, A'8-A'9, A'9-A'10, A'10-A'11, A'11-A'12, A'12-A'13, A'13-A'14, B'1-B'2, B'2-B'3, B'3-B'4, B'4-B'5, B'5-B'6, B'6-B'7, B'7-B'8, B'8-B'9, B'9-B'10, B'10-B'11, B'11-B'12, C'1-C'2, C'2-C'3, C'3-C'4, C'4-C'5, C'5-C'6, C'6-C'7, C'7-C'8, D'1-D'2, D'2-D'3, D'3-D'4, D'4-D'5, D'5-D'6, D'6-D'7, D'7-D'8, E'1-E'2, E'2-E'3, E'3-E'4, E'4-E'5, E'5-E'6, E'6-E'7, E'7-E'8, E'8-E'9, E'9-E'10, F'1-F'2, F'2-F'3, F'3-F'4, F'4-F'5, F'5-F'6, F'6-F'7, G'1-G'2, G'2-G'3, G'3-G'4, G'4-G'5, G'5-G'6, G'6-G'7, G'7-G'8, G'8-G'9, G'9-G'10, G'10-G'11, G'11-G'12, G'12-G'13, G'13-G'14, G'14-G'15, G'15-G'16, H'1-H'2, H'2-H'3, H'3-H'4, H'4-H'5, H'5-H'6, H'6-H'7, H'7-H'8, H'8-H'9, H'9-H'10, H'

/tmp/ipykernel_3348/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif
[done]
Reading 0 ... 3059399  =      0.000 ...  1529.699 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=173, n_times=3059400
    Range : 0 ... 3059399 =      0.000 ...  1529.699 secs
Ready.
Added the following bipolar channels:
A'1-A'2, A'2-A'3, A'3-A'4, A'4-A'5, A'5-A'6, A'6-A'7, A'7-A'8, A'8-A'9, A'9-A'10, A'10-A'11, A'11-A'12, A'12-A'13, A'13-A'14, B'1-B'2, B'2-B'3, B'3-B'4, B'4-B'5, B'5-B'6, B'6-B'7, B'7-B'8, B'8-B'9, B'9-B'10, B'10-B'11, B'11-B'12, C'1-C'2, C'2-C'3, C'3-C'4, C'4-C'5, C'5-C'6, C'6-C'7, C'7-C'8, D'1-D'2, D'2-D'3, D'3-D'4, D'4-D'5, D'5-D'6, D'6-D'7, D'7-D'8, E'1-E'2, E'2-E'3, E'3-E'4, E'4-E'5, E'5-E'6, E'6-E'7, E'7-E'8, E'8

/tmp/ipykernel_3348/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_raw_clean_data.fif
[done]


In [8]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 160, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.


/tmp/ipykernel_3348/2669736035.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]


Channels marked as bad:
['C3', 'C4', 'CZ', 'EKG1', 'EKG2', 'EMPTY', 'EMPTY_191', 'EMPTY_192', 'EMPTY_2', 'EMPTY_205', 'EMPTY_206', 'EMPTY_207', 'EMPTY_208', 'EMPTY_209', 'EMPTY_210', 'EMPTY_211', 'EMPTY_212', 'EMPTY_213', 'EMPTY_214', 'EMPTY_215', 'EMPTY_216', 'EMPTY_217', 'EMPTY_218', 'EMPTY_219', 'EMPTY_220', 'EMPTY_221', 'EMPTY_222', 'EMPTY_223', 'EMPTY_224', 'EMPTY_225', 'EMPTY_226', 'EMPTY_227', 'EMPTY_228', 'EMPTY_229', 'EMPTY_230', 'EMPTY_231', 'EMPTY_3', 'EMPTY_4', 'EMPTY_59', 'EMPTY_60', 'EMPTY_61', 'EMPTY_62', 'EMPTY_63', 'EMPTY_64', "F'8", 'F10', 'F3', 'F4', 'F7', 'F8', 'F9', 'FP1', 'FP2', 'FPZ', 'FZ', "K'14", 'O1', 'O2', 'OZ', 'P3', 'P4', 'P7', 'P8', 'PZ', 'REF1', 'REF2', 'T7', 'T8']


## Epoching the data 

### Onset

In [9]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.


/tmp/ipykernel_3348/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...


/tmp/ipykernel_3348/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


### Trial End

In [10]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.


/tmp/ipykernel_3348/749539899.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_3348/749539899.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...


### Last Away

In [3]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,16,14300,0,1
1,2,14,35600,0,1
2,3,15,46000,0,1
3,4,9,54700,0,1
4,5,5,66100,0,1
...,...,...,...,...,...
208,233,18,2891500,0,1
209,234,7,2901900,0,1
210,236,17,2962500,0,1
211,237,14,2977700,0,1


In [12]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [13]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.
Not setting metadata
213 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 107 events and 20001 original time points ...


/tmp/ipykernel_3348/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_3348/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 106 events and 20001 original time points ...


### First Dot

In [14]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,16,9800,0,1
1,1,13,22000,0,1
2,2,14,31700,0,1
3,3,15,44800,0,1
4,5,5,60500,0,1
...,...,...,...,...,...
226,235,3,2910300,0,1
227,236,17,2960500,0,1
228,237,14,2973000,0,1
229,238,12,2987400,0,1


In [15]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [16]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.
Not setting metadata
231 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 116 events and 18001 original time points ...


/tmp/ipykernel_3348/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_3348/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 115 events and 18001 original time points ...


## First Move

In [4]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,16,9500,0,1
1,1,13,21200,0,1
2,2,14,30800,0,1
3,3,15,44300,0,1
4,4,9,53100,0,1
...,...,...,...,...,...
234,235,3,2909900,0,1
235,236,17,2959700,0,1
236,237,14,2972200,0,1
237,238,12,2987200,0,1


In [5]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [11]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_76980/2674478381.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


## Ghost Attack

In [22]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,13,22800,0,1
1,3,15,45500,0,1
2,6,3,85900,0,1
3,7,10,95600,0,1
4,10,2,131500,0,1
5,17,4,222700,0,1
6,18,8,248300,0,1
7,24,2,322400,0,1
8,30,5,401700,0,1
9,37,11,478700,0,1


In [23]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [24]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif...
    Range : 0 ... 2215999 =      0.000 ...  1107.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data-1.fif...
    Range : 2216000 ... 3059399 =   1108.000 ...  1529.699 secs
Ready.
Not setting metadata
59 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 59 events and 20001 original time points ...


/tmp/ipykernel_3348/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_3348/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH029/ieeg/BJH029_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
